# HYBRID SEARCH

In [1]:
pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('PINECONE_API_KEY')

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
## responsible in doing both sematic search (dense vectors) and keyword search ( sparse vectors)

In [4]:
from pinecone import Pinecone,ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

## initialize the pinecone client
pc = Pinecone(api_key)

## create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension= 384, ## dimension of dense vector 
        metric= 'dotproduct',  ## sparse values supported only for dotproduct
        spec = ServerlessSpec(cloud='aws',region='us-east-1')
        
    )

In [5]:
index = pc.Index(index_name)
index

In [ ]:
## vector embedding 
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [8]:
embedding

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
## sparse embedding
from pinecone_text.sparse import BM25Encoder ## used tf-idf

encoder = BM25Encoder().default()  ## default is tf-idf
encoder

In [10]:
sentences = [
  "In 1913, the world's first crossword puzzle was published in the New York World newspaper.",
  "In Rust, the borrow checker ensures memory safety without a garbage collector.",
  "In the depths of the Mariana Trench, pressure exceeds 1000 times that at sea level."
]

## tf-idf values on these sentences
encoder.fit(sentences)

## store the values to a JSON file
encoder.dump("bm25_values.json")

## load to your BM25Encoder object
encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
encoder

In [18]:
retriever = PineconeHybridSearchRetriever(embeddings=embedding,sparse_encoder=encoder,index=index)

In [19]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000023C824455D0>, index=<pinecone.db_data.index.Index object at 0x0000023CBB77EA40>)

In [20]:
retriever.add_texts(
    [
        "In 1913, the world's first crossword puzzle was published in the New York World newspaper.",
  "In Rust, the borrow checker ensures memory safety without a garbage collector.",
  "In the depths of the Mariana Trench, pressure exceeds 1000 times that at sea level."
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
retriever.invoke("when and where was the world's first crossword puzzle published?")

[Document(metadata={'score': 0.621355891}, page_content="In 1913, the world's first crossword puzzle was published in the New York World newspaper."),
 Document(metadata={'score': 0.0489597321}, page_content='In the depths of the Mariana Trench, pressure exceeds 1000 times that at sea level.'),
 Document(metadata={'score': 0.0127863884}, page_content='In Rust, the borrow checker ensures memory safety without a garbage collector.')]